In [21]:
import os
import pandas as pd
import load_geolife
import add_geohash
import create_nodes
import geohash_location_count
import geohash_user_count
import create_geohash_meet
import create_geohash_meet_time

In [14]:
def create_dirs():
    if not os.path.isdir('images/'):
        os.makedirs('images/')
    if not os.path.isdir('htmls/'):
        os.makedirs('htmls/')
    if not os.path.isdir('data/'):
        os.makedirs('data/')

In [15]:
def removeDuplicates():
    duplicates = pd.read_csv('data/geolife_duplicates_content.csv')
    df = pd.read_csv('data/geolife_geohash_size_8.csv')
    df['px'] = df['Person ID'].astype('string') + "_" + df['Trajectory'].astype('string') + ".plt"
    duplicates['px'] = duplicates['user'].astype('string')+"_"+duplicates['file_name'].astype('string')
    noduplicates = df[~df['px'].isin(duplicates['px'])].copy(deep=True)
    noduplicates.to_csv('data/geolife_geohash_size_8_no_duplicates.csv',index=False)


In [23]:
create_dirs()
load_geolife.load_geolife('./geolife/Data/')
add_geohash.add_geohash()

removeDuplicates()
create_nodes.create_nodes()
geohash_location_count.geohash_location_count()
geohash_user_count.geohash_user_count()
count_files = create_geohash_meet.createGeohashMeetJsons()
create_geohash_meet.createGeohashMeetCSV(count_files)

create_geohash_meet_time.create_geohash_time_meets()

AttributeError: module 'create_geohash_meet_time' has no attribute 'create_geohash_time_meets'